# **Web Scrapping Cuevana**

In [41]:
import re
import colorama
from colorama import Fore
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

**Codigo de Web Scrapping**

In [146]:
url='https://cuevana.biz/genero/misterio/page/'
for pagina in range(1, 3):
    url_pagina = url + str(pagina)
    print(url_pagina)

https://cuevana.biz/genero/misterio/page/1
https://cuevana.biz/genero/misterio/page/2


In [184]:
url_pagina='https://cuevana.biz/genero/misterio/page/'
titles=[]
descripcions=[]
fechas=[]
generos=[]
minutos=[]
ranks=[]
for pagina in range(1,10):
    url=url_pagina + str(pagina)
    resultado=requests.get(url)
    content=resultado.text
    soup = BeautifulSoup(content, 'html.parser')
    fuente=soup.find_all('li',class_='TPostMv')
    for elemento in fuente:
        titulos=elemento.find_all('div',class_='Title')
        descripciones=elemento.find_all('div',class_='Description')
        extrenos=elemento.find_all('span',class_='Date AAIco-date_range')
        duraciones=elemento.find_all('span',class_='Time AAIco-access_time')
        rankings=elemento.find_all('span',class_='Vote AAIco-star')
        for titulo in titulos:
            title_text=titulo.get_text(strip=True)
            titles.append(title_text)
        for descrip in descripciones:
            descripcion_texto=descrip.get_text(strip=True)
            descripcions.append(descripcion_texto)
            genero_span=descrip.find('p',class_='Genre AAIco-movie_creation')
            for gene in genero_span:
                genre_text=gene.get_text(strip=True)
                generos.append(genre_text)
        for extre in extrenos:
            extre_text=extre.get_text(strip=True)
            fechas.append(extre_text)
        for duracion in duraciones:
            duracion_title=duracion.get_text(strip=True)
            minutos.append(duracion_title)
        for rank in rankings:
            rank_title=rank.get_text(strip=True)
            ranks.append(rank_title)

In [185]:
# hay datos vacios en los minutos en la pagina 1
# hay datos vacioes en los rankings en la pagina 3
print(f'Total de datos en los titulos {len(titles)}')
print(f'Total de datos en las descripciones {len(descripcions)}')
print(f'Total de datos en las fechas {len(fechas)}')
print(f'Total de datos en las generos {len(generos)}')
print(f'Total de datos en los minutos {len(minutos)}')
print(f'Total de datos en las rankings {len(ranks)}')

Total de datos en los titulos 216
Total de datos en las descripciones 216
Total de datos en las fechas 216
Total de datos en las generos 2868
Total de datos en los minutos 215
Total de datos en las rankings 215


In [186]:
print('----------------------------------------------------')
if len(minutos)!=len(titles):
    minutos.insert(4,'0h')
    ranks.insert(53,0)
print(f'Total de datos en los titulos {len(titles)}')
print(f'Total de datos en las descripciones {len(descripcions)}')
print(f'Total de datos en las fechas {len(fechas)}')
print(f'Total de datos en las generos {len(generos)}')
print(f'Total de datos en los minutos {len(minutos)}')
print(f'Total de datos en las rankings {len(ranks)}')

----------------------------------------------------
Total de datos en los titulos 216
Total de datos en las descripciones 216
Total de datos en las fechas 216
Total de datos en las generos 2868
Total de datos en los minutos 216
Total de datos en las rankings 216


In [187]:
# Suponiendo que tienes un DataFrame llamado df_gen
data_generos = {'Generos': generos}
df_gen = pd.DataFrame(data_generos)
df_gen.dropna(subset=['Generos'], inplace=True)
# Suponiendo que tienes un DataFrame llamado df_gen
df_gen = df_gen[(df_gen['Generos'] != ',') & (df_gen['Generos'] != '')]
print(df_gen['Generos'].value_counts())

Generos
Género:            216
Misterio           216
Suspenso           134
Drama               76
Terror              72
Crimen              43
Ciencia ficción     36
Acción              29
Comedia             29
Animación           20
Familia             19
Aventura            17
Fantasía            13
Romance              7
Western              2
Historia             2
Música               1
Guerra               1
Name: count, dtype: int64


In [188]:
# Supongamos que tienes un DataFrame llamado df_gen
# Crear una lista para almacenar los nuevos datos
nuevos_datos = []

# Inicializar variables para almacenar los valores de género y tipo de género
genero_actual = None
tipo_genero = []

# Iterar sobre las filas del DataFrame
for index, row in df_gen.iterrows():
    # Verificar si el valor de la columna 'Generos' es 'Género:'
    if row['Generos'] == 'Género:':
        # Si es así, guardar los valores actuales y reiniciar la lista de tipo de género
        if genero_actual is not None:
            nuevos_datos.append({'Género': genero_actual, 'Tipo de Género': ', '.join(tipo_genero)})
        genero_actual = row['Generos']
        tipo_genero = []
    else:
        # Si no, agregar el valor a la lista de tipo de género
        tipo_genero.append(row['Generos'])

# Añadir el último conjunto de datos
if genero_actual is not None:
    nuevos_datos.append({'Género': genero_actual, 'Tipo de Género': ', '.join(tipo_genero)})

# Crear un nuevo DataFrame a partir de los nuevos datos
nuevo_df = pd.DataFrame(nuevos_datos)

print(nuevo_df)


      Género                                Tipo de Género
0    Género:                    Terror, Misterio, Suspenso
1    Género:  Acción, Suspenso, Misterio, Crimen, Aventura
2    Género:             Drama, Terror, Misterio, Suspenso
3    Género:             Terror, Misterio, Suspenso, Drama
4    Género:                    Suspenso, Terror, Misterio
..       ...                                           ...
211  Género:                              Terror, Misterio
212  Género:                              Terror, Misterio
213  Género:             Acción, Comedia, Crimen, Misterio
214  Género:             Ciencia ficción, Terror, Misterio
215  Género:                    Terror, Misterio, Suspenso

[216 rows x 2 columns]


In [189]:
# Crear DataFrame con los datos
data = {'Pelicula': titles, 'Fecha de Estreno': fechas, 'Descripcion':descripcions, 'Duracion':minutos, 'Rankings':ranks}
df = pd.DataFrame(data)
df['Genero']=nuevo_df['Tipo de Género']
df

,Pelicula,Fecha de Estreno,Descripcion,Duracion,Rankings,Genero
0,Slender Man,2018,“El hombre más pálido. El traje más oscuro. Má...,1h 33min,4.31,"Terror, Misterio, Suspenso"
1,Plan de Escape 2,2018,El experto en seguridad Ray Breslin Sylvester ...,1h 45min,5.16,"Acción, Suspenso, Misterio, Crimen, Aventura"
2,Revelaciones,2000,Norman y Claire son un matrimonio normal: él e...,2h 03min,6.41,"Drama, Terror, Misterio, Suspenso"
3,Apóstol,2018,Thomas Richardson llega de incógnito a una isl...,2h 10min,6.1,"Terror, Misterio, Suspenso, Drama"
4,La Niña De La Mina,2016,"En Guanajuato, después de que una chica desapa...",0h,6.1,"Suspenso, Terror, Misterio"
...,...,...,...,...,...,...
211,Los Niños del Maíz 666: Cosecha negra,1999,Hannah Martin está viajando en busca de su mad...,1h 22min,4.6,"Terror, Misterio"
212,El aro,2002,Parecía simplemente otra leyenda urbana: una c...,1h 55min,6.67,"Terror, Misterio"
213,Un espía y medio,2016,Después de que se reconecta con un amigo raro ...,1h 47min,6.41,"Acción, Comedia, Crimen, Misterio"
214,Strange Behavior,1981,En una pequeña localidad Americana han tenido ...,1h 39min,5.4,"Ciencia ficción, Terror, Misterio"
